In [14]:
import pandas as pd
import re
import pickle
import numpy as np
import pandas as pd

# plotting
import seaborn as sns
# from wordcloud import WordCloud
import matplotlib.pyplot as plt

# nltk
from nltk.stem import WordNetLemmatizer


In [136]:
df = pd.read_csv('../files/twitter_tweets/tweets_#SarTanSeJuda.csv')

output_csv = r'../files/cleaned_csvs/tweets_#SarTanSeJuda_cleaned.csv'

In [137]:
df.head()

,Unnamed: 0,user_id,username,id,date,Tweet,lang,replyCount,followersCount
0,0,80183148,skgrb,1569601097844621312,2022-09-13 08:17:18+00:00,openly promoting #lovejihad #Sartansejuda twit...,en,0,198
1,1,1552629627210039297,Anand2608Singh,1569598383710674944,2022-09-13 08:06:31+00:00,@_sayema What about the who sing #Sartansejuda...,in,0,1
2,2,1542916777096904704,AkhilBha58,1569591986826498051,2022-09-13 07:41:05+00:00,8 saal ka jhol japat JholaChaap Madari modi ka...,hi,0,385
3,3,34245009,indiatvnews,1569590452747370497,2022-09-13 07:35:00+00:00,दोपहर प्राइम टाइम \n\nदेखिए इस वक्त की टॉप-10 ...,hi,1,2118287
4,4,706820958741356544,IndiaTVHindi,1569590452692860928,2022-09-13 07:35:00+00:00,दोपहर प्राइम टाइम \n\nदेखिए इस वक्त की टॉप-10 ...,hi,0,91130


In [138]:
df.shape

(6785, 9)

In [139]:
df_eng = df[df.lang == 'en']
df_eng.head()

,Unnamed: 0,user_id,username,id,date,Tweet,lang,replyCount,followersCount
0,0,80183148,skgrb,1569601097844621312,2022-09-13 08:17:18+00:00,openly promoting #lovejihad #Sartansejuda twit...,en,0,198
11,11,1524779594372775936,Journalist_ZAK,1569583480258052097,2022-09-13 07:07:17+00:00,"Kashi Mosque case Verdict, is the booster of b...",en,0,2
14,14,715773652,TheAllenVictor,1569576021057490944,2022-09-13 06:37:39+00:00,It's like u r not able to bear with it.\n\nHow...,en,1,420
17,17,1496033252611866625,manishagrawal_7,1569571942398840832,2022-09-13 06:21:26+00:00,@ShashiTharoor @kunalkamra88 @virdas @munawar0...,en,0,17
19,19,386452694,SinghVijukum,1569565685503922176,2022-09-13 05:56:35+00:00,No more will you accuse the #BangalPolice and ...,en,0,160


In [140]:
df.shape, df_eng.shape

((6785, 9), (4559, 9))

In [141]:
df_eng['user_id'].nunique(), df['user_id'].nunique()

(2273, 3349)

In [142]:
# Defining dictionary containing all emojis with their meanings.
emojis = {':)': 'smile', ':-)': 'smile', ';d': 'wink', ':-E': 'vampire', ':(': 'sad', 
          ':-(': 'sad', ':-<': 'sad', ':P': 'raspberry', ':O': 'surprised',
          ':-@': 'shocked', ':@': 'shocked',':-$': 'confused', ':\\': 'annoyed', 
          ':#': 'mute', ':X': 'mute', ':^)': 'smile', ':-&': 'confused', '$_$': 'greedy',
          '@@': 'eyeroll', ':-!': 'confused', ':-D': 'smile', ':-0': 'yell', 'O.o': 'confused',
          '<(-_-)>': 'robot', 'd[-_-]b': 'dj', ":'-)": 'sadsmile', ';)': 'wink', 
          ';-)': 'wink', 'O:-)': 'angel','O*-)': 'angel','(:-D': 'gossip', '=^.^=': 'cat'}

## Defining set containing all stopwords in english.
stopwordlist = ['a', 'about', 'above', 'after', 'again', 'ain', 'all', 'am', 'an',
             'and','any','are', 'as', 'at', 'be', 'because', 'been', 'before',
             'being', 'below', 'between','both', 'by', 'can', 'd', 'did', 'do',
             'does', 'doing', 'down', 'during', 'each','few', 'for', 'from', 
             'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here',
             'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in',
             'into','is', 'it', 'its', 'itself', 'just', 'll', 'm', 'ma',
             'me', 'more', 'most','my', 'myself', 'now', 'o', 'of', 'on', 'once',
             'only', 'or', 'other', 'our', 'ours','ourselves', 'out', 'own', 're',
             's', 'same', 'she', "shes", 'should', "shouldve",'so', 'some', 'such',
             't', 'than', 'that', "thatll", 'the', 'their', 'theirs', 'them',
             'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 
             'through', 'to', 'too','under', 'until', 'up', 've', 'very', 'was',
             'we', 'were', 'what', 'when', 'where','which','while', 'who', 'whom',
             'why', 'will', 'with', 'won', 'y', 'you', "youd","youll", "youre",
             "youve", 'your', 'yours', 'yourself', 'yourselves']

In [143]:
def preprocess(textdata):
    processedText = []
    
    # Create Lemmatizer and Stemmer.
    wordLemm = WordNetLemmatizer()
    
    # Defining regex patterns.
    urlPattern        = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
    userPattern       = '@[^\s]+'
    hashtagPattern    = '#[^\s]+'
    alphaPattern      = "[^a-zA-Z0-9]"
    sequencePattern   = r"(.)\1\1+"
    seqReplacePattern = r"\1\1"
    
    for tweet in textdata:
        tweet = tweet.lower()
        
        # Replace all URls with 'URL'
        tweet = re.sub(urlPattern,' URL',tweet)
        # Replace all emojis.
        for emoji in emojis.keys():
            tweet = tweet.replace(emoji, "EMOJI" + emojis[emoji])        
        # Replace @USERNAME to 'USER'.
        tweet = re.sub(userPattern,' ', tweet)   
        
        tweet = re.sub(hashtagPattern, ' ', tweet)
        
        # Replace all non alphabets.
        tweet = re.sub(alphaPattern, " ", tweet)
        # Replace 3 or more consecutive letters by 2 letter.
        tweet = re.sub(sequencePattern, seqReplacePattern, tweet)

        tweetwords = ''
        for word in tweet.split():
            # Checking if the word is a stopword.
            #if word not in stopwordlist:
            if len(word)>1:
                # Lemmatizing the word.
                word = wordLemm.lemmatize(word)
                tweetwords += (word+' ')
            
        processedText.append(tweetwords)
        
    return processedText

In [144]:
import time
text = list(df_eng['Tweet'])
t = time.time()
processedtext = preprocess(text)
print(f'Text Preprocessing complete.')
print(f'Time Taken: {round(time.time()-t)} seconds')

Text Preprocessing complete.
Time Taken: 1 seconds


In [145]:
df_eng['preprocessed_text'] = processedtext

<ipython-input-145-fa529211b090>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eng['preprocessed_text'] = processedtext


In [146]:
df_eng.head(30)

,Unnamed: 0,user_id,username,id,date,Tweet,lang,replyCount,followersCount,preprocessed_text
0,0,80183148,skgrb,1569601097844621312,2022-09-13 08:17:18+00:00,openly promoting #lovejihad #Sartansejuda twit...,en,0,198,openly promoting twitter com swati g statu
11,11,1524779594372775936,Journalist_ZAK,1569583480258052097,2022-09-13 07:07:17+00:00,"Kashi Mosque case Verdict, is the booster of b...",en,0,2,kashi mosque case verdict is the booster of bi...
14,14,715773652,TheAllenVictor,1569576021057490944,2022-09-13 06:37:39+00:00,It's like u r not able to bear with it.\n\nHow...,en,1,420,it like not able to bear with it how about ur ...
17,17,1496033252611866625,manishagrawal_7,1569571942398840832,2022-09-13 06:21:26+00:00,@ShashiTharoor @kunalkamra88 @virdas @munawar0...,en,0,17,mr tharrur the right to offend can be one side...
19,19,386452694,SinghVijukum,1569565685503922176,2022-09-13 05:56:35+00:00,No more will you accuse the #BangalPolice and ...,en,0,160,no more will you accuse the and of complacency...
22,22,1466237648113078272,WokeLingam,1569551402179117057,2022-09-13 04:59:49+00:00,@KeypadGuerilla @Sanjay_Dixit This is happenin...,en,0,15,this is happening because muslim are provoking...
23,23,1494942546430029824,of_dikshit,1569546473242771458,2022-09-13 04:40:14+00:00,"Why only apology \n\nWe lost tens, in #SarTanS...",en,0,105,why only apology we lost ten in frenzy those w...
24,24,1144508523679645696,LegalLro,1569538642804359168,2022-09-13 04:09:07+00:00,Tribal youth Dipak Barde proven killed for mar...,en,7,48850,tribal youth dipak barde proven killed for mar...
25,25,306357166,SimplyyJKS,1569526839651217409,2022-09-13 03:22:13+00:00,"@AjeetBhartii Gustakh-e-nabi ki ek saza,\n#Sar...",en,0,32,gustakh nabi ki ek saza ye kya it direct threa...
26,26,306357166,SimplyyJKS,1569526770210308096,2022-09-13 03:21:57+00:00,"Gustakh-e-nabi ki ek saza,\n#Sartansejuda #Sar...",en,0,32,gustakh nabi ki ek saza ye kya it direct threa...


In [147]:
df_eng.to_csv(output_csv, index = False, header=True)
